# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Introduction

### Background 

New York City's demographics show that it is a large and ethnically diverse metropolis. It is the largest city in the United States with a long history of international immigration. New York City was home to nearly 8.5 million people in 2014. 
While English remains the most widely spoken language, there are areas in the outer boroughs in which up to 25% of people speak English only as an alternate language. 

Indians are the largest South Asian group, comprising 2.4% of the city's population, among the largest in the US cities. New york city also sees a lot of tourists consuming at local buissness. At 65.2 million the number of tourists is many times the population of the city at 8.1 million. New york city also sees a lot of commute from outside the city. Tourists and commuters indicate that residential data alone might not be a good indicator of the consumption patterns in the city. All of these factors make New York city a unique problem for data analysis. 

Indian are highly popular in the united states. These restaurants attract a diverse set of people, and not just Indian origin people. This suggests factors other than just ethnicity of the poeple wouldn't dictate the presence of an Indian restaurant. Determining the reason for the presence of an Indian restaurant would open up data savvy entreprenuer to find neighborhoods to open an Indian restaurant.  


### Buisness Problem: Determine the variable that can predict the presence of Indian restaurants in NYC

## Data section

The data used would two sets of categories the Target and the paramters
The Target data would be the number of restaurants per NYC borough. This would be obtained from foursqaure API
The data for the parameters would be 

    1) People of Indian and other south asian countries Origin from https://www.baruch.cuny.edu/nycdata/population-geography/foreign-birthcountry.htm
    2) Buissnesses of other categories such as Chinese restaurants, Yoga studios etc from fourrsquare
    3) General population statistic data https://guides.newman.baruch.cuny.edu/nyc_data/nbhoods
        Social
        Economic
        Housing
        Demographic
The idead is to find correlation between number of indian restaurants and any of the parameters. 

Interpolation techniques would be used to find the relationship and the strength of the relationship 

### Data scraping and cleaning
Loading libraries 

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 

Load Neighboorhood data

In [6]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Download indian population statistics

In [24]:
#data from https://www.baruch.cuny.edu/nycdata/population-geography/foreign-birthcountry.htm
popu = [2233, 9335, 15992, 48249,3387]
totPopu = [535582, 959025, 470212, 1115696, 114425]
boroughs = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

# create empty data frame in pandas
dfInds = pd.DataFrame()
dfInds['borough'] = boroughs
dfInds['population'] = popu
dfInds['totalPopulation'] = totPopu
dfInds.head()

,borough,population,totalPopulation
0,Bronx,2233,535582
1,Brooklyn,9335,959025
2,Manhattan,15992,470212
3,Queens,48249,1115696
4,Staten Island,3387,114425


In [ ]:
CLIENT_ID = 'ORCWPBRVJMO5Y2P5SNEGUDEM5PLAU4E1CM130VRDGYBBHRN5' # your Foursquare ID
CLIENT_SECRET = 'VHKPV30MAVOCTZCY12O4PUKWFLS1GSCHZRYXXC5IOOL2DKQV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
#results
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# convert into a dataframe 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()